In [ ]:
import os
import re
import json
import time

from src.utils import (
    get_params,
    getNewOrderNotification,
    getLateOrderNotification
)

from src.bling_classes import (
    BlingAPI,
    BlingAuth,
    BlingDfTreat,
    BlingRequests
)

In [ ]:
cwd = os.getcwd()
resources = os.path.join(cwd,'resources')

In [ ]:


params = get_params()
autoModifySituations = params['autoModifySituations']
situationsToGetLogistics = params['situationsToGetLogistics']
situationToNotify = params['situationToNotify']

if autoModifySituations:
    verification = input('Aviso: O update automático das situações de pedido está ativado, deseja prosseguir? [s/n]: ')
    
    if verification == 's':
        pass
    else:
        exit()

late_param = params['late_param']
late_timerLoop = params['late_timerLoop']

if not(type(situationsToGetLogistics) == list):
    print('situationsToGetLogistics deve ser uma lista')
    input('Pressione ENTER para sair.')
    exit()

if not(type(situationToNotify) == dict):
    print('situationToNotify deve ser uma lista')
    input('Pressione ENTER para sair.')
    exit()
# print(situationToNotify)

In [ ]:
waitingOrdersDict = {}
minutes_running = 0
late_timer = 0

testloop = True
loop_interval = (60 - 4)

while testloop:
    try:
        bling = BlingDfTreat(BlingRequests.rawOrdersDf())
        bling.selectIdOnSituationColumn()
        bling.translateIdOnSituationColumn()
        bling.selectNameOnContactColumn()
        bling.addLogisticColumn(situationsToGetLogistics)
        bling.translateLogisticId()

    except:
        print('error fetching data from API')
        BlingAuth.updateBlingToken()
        print('test')
        time.sleep(1)
        continue
    
    if autoModifySituations:
        bling.autoUpdateSituations()
        
    waitingOrdersMask = bling.dataframe.situacao.str.contains('|'.join(situationToNotify.keys()))
    waitingOrdersDf = bling.dataframe.loc[waitingOrdersMask]

    if waitingOrdersMask.any():
        for i,v in waitingOrdersDf.iterrows():
            if i not in waitingOrdersDict:
                waitingOrdersDict[i] = {
                    'num':v.numero 
                    ,'situation':v.situacao
                    ,'logistic':v.logistics 
                    ,'count':0
                } 
            else:
                waitingOrdersDict[i]['situation'] = v.situacao
                waitingOrdersDict[i]['logistic'] = v.logistics
                waitingOrdersDict[i]['count'] += 1
        
         # removes attended orders
        for i in list(waitingOrdersDict.keys()):
            if  i not in waitingOrdersDf.index:
                waitingOrdersDict.pop(i)
    else:
        waitingOrdersDict = {}

    os.system('cls')

    for situation in situationToNotify.keys():
        ordersBySituation_dict = {k:v for k,v in waitingOrdersDict.items() if re.search(situation,v['situation'])}

        new = [ordersBySituation_dict[i]['num'] for i in ordersBySituation_dict if ordersBySituation_dict[i]['count'] == 0]
        waiting = [ordersBySituation_dict[i]['num'] for i in ordersBySituation_dict\
                if (0 < ordersBySituation_dict[i]['count'] < late_param)]
        late = [ordersBySituation_dict[i]['num'] for i in ordersBySituation_dict if ordersBySituation_dict[i]['count'] >= late_param]

        print(situation.replace('\\',''))
        print('new orders: ', new)
        print('order waiting: ', waiting)
        print(f'orders waiting for {late_param}min or more: ', late,'\n')
        print('#---------------------------------------------------#\n')

    print(f'program running for: {minutes_running} minutes\n')
    minutes_running += 1

    # dict used for notifications
    new = [waitingOrdersDict[i]['num'] for i in waitingOrdersDict if waitingOrdersDict[i]['count'] == 0 and\
            situationToNotify[waitingOrdersDict[i]['situation']\
                .replace('(','\(').replace(')','\)')]\
                    ['notifyNew']]
    
    waiting = [waitingOrdersDict[i]['num'] for i in waitingOrdersDict\
            if (0 < waitingOrdersDict[i]['count'] < late_param)]
    
    late = [waitingOrdersDict[i]['num'] for i in waitingOrdersDict\
            if waitingOrdersDict[i]['count'] >= late_param and\
                situationToNotify[waitingOrdersDict[i]['situation']\
                    .replace('(','\(').replace(')','\)')]\
                        ['notifyOld']]
    
    if new:
        getNewOrderNotification(new)
        late_timer = 0

    elif late and not waiting:
        if late_timer == 0 or late_timer == late_timerLoop:
            late_timer = 1
            getLateOrderNotification(late)
        elif late_timer < late_timerLoop:
            late_timer += 1
    
    # testloop = False
    time.sleep(loop_interval)
